In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readData(name):
    train = pd.read_csv(name+".csv")
    train=train.drop(["date"], axis=1)
    train=train.drop(["Unnamed: 0"], axis=1)
    return train

def sta(train):
    train=train.convert_objects(convert_numeric=True)
    train= train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train

def buildTrain(train, pastDay, futureDay):
    X_train, Y_train ,Z_train= [], [],[]
    for i in range(train.shape[0]-futureDay-pastDay+1):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        a=np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["high"])
        b=np.array(train.iloc[i+pastDay-1:i+pastDay+futureDay-1]["high"])
        if a>=b:
            Y_train.append(1)
        else:
            Y_train.append(0)                                                       
    return np.array(X_train), np.array(Y_train)

def shuffle(X,Y):
    np.random.seed()
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

def buildModel(train_x,train_y,bs):
    model = Sequential()
    model.add(LSTM(32, input_length=train_x.shape[1],input_dim= train_x.shape[2],return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    model.summary()
    callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
    model.fit(train_x,train_y, epochs=1000, batch_size=bs, validation_split=0.1, callbacks=[callback])
    return model,[32,32,0,0,0]



In [ ]:
lookback=7
batch_size=32
col_name=["1","2","3","4","5","acc","lookback","batch_size","name","num"]
place_name=["train+oil+weather+國定假日+拜拜日 雲林","train+oil+weather+國定假日+拜拜日 嘉義","train+oil+weather+國定假日+拜拜日 彰化","train+oil+weather+國定假日+拜拜日 台南","train+oil+weather+國定假日+拜拜日 高雄","train+oil+weather+國定假日+拜拜日 屏東","train+oil+weather+國定假日+拜拜日 台中","train+oil+weather+國定假日+拜拜日 苗栗","train+oil+weather+國定假日+拜拜日 桃園","train+oil+weather+國定假日+拜拜日 台北"]
for i in place_name:
    df=pd.DataFrame(columns=col_name)
    data=[]
    for j in range(5):
        train=readData(i)
        temp=train
        train=sta(train)
        train_x1,train_y1=buildTrain(train,lookback,1)
        train_x2,train_y2=buildTrain(temp,lookback,1)
        train_x,train_y=train_x1,train_y2
        train_x,train_y= shuffle(train_x,train_y)
        train_x,train_y, val_x, val_y= splitData(train_x,train_y, 0.05)
        model,layer=buildModel(train_x,train_y,batch_size)
        pre=model.evaluate(val_x,val_y,verbose=0)
        data.append({"1":layer[0],"2":layer[1],"3":layer[2],"4":layer[3],"5":layer[4],"acc":pre[1]*100,"lookback":lookback,"batch_size":batch_size,"name":i,"num":j})
        
    df=pd.concat([pd.DataFrame(data), df], ignore_index=True,sort=True)
    df.to_csv('class'+i+'_data'+'(lookback  '+str(lookback)+')'+'(bs  '+str(batch_size)+')'+'.csv', encoding='utf_8_sig')

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:39: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, input_shape=(7, 27), return_sequences=True)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 32)             7680      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 16,033
Trainable params: 16,033
Non-trainable params: 0
_________________________________________________________________
Train on 2257 samples, validate on 251 samples
Epoch 1/100
2257/2257 [==============================] - 2s 828us/step - loss: 0.6924 - acc: 0.5224 - val_loss: 0.6929 - val_acc: 0.4861
Epoch 2/100
2257/2257 [==============================] - 0s 218us/step - loss: 0.6869 - acc: 0.5348 - val_loss: 0.6862 - val_acc: 0.5418
Epoch 3/100
2257/2257 [==============================] - 0s 202us/step - loss: 0.

Epoch 55/100
2257/2257 [==============================] - 1s 223us/step - loss: 0.3538 - acc: 0.8259 - val_loss: 0.5654 - val_acc: 0.7131
Epoch 56/100
2257/2257 [==============================] - 1s 224us/step - loss: 0.3511 - acc: 0.8268 - val_loss: 0.6065 - val_acc: 0.6892
Epoch 57/100
2257/2257 [==============================] - 1s 224us/step - loss: 0.3419 - acc: 0.8352 - val_loss: 0.5711 - val_acc: 0.7291
Epoch 58/100
2257/2257 [==============================] - 0s 211us/step - loss: 0.3372 - acc: 0.8418 - val_loss: 0.5951 - val_acc: 0.7211
Epoch 59/100
2257/2257 [==============================] - 0s 205us/step - loss: 0.3337 - acc: 0.8374 - val_loss: 0.6488 - val_acc: 0.7131
Epoch 60/100
2257/2257 [==============================] - 0s 216us/step - loss: 0.3235 - acc: 0.8480 - val_loss: 0.6087 - val_acc: 0.7052
Epoch 61/100
2257/2257 [==============================] - 1s 234us/step - loss: 0.3163 - acc: 0.8525 - val_loss: 0.6690 - val_acc: 0.7052
Epoch 62/100
2257/2257 [==========

2257/2257 [==============================] - 0s 203us/step - loss: 0.2318 - acc: 0.9025 - val_loss: 0.7700 - val_acc: 0.7211
Epoch 68/100
2257/2257 [==============================] - 0s 219us/step - loss: 0.2255 - acc: 0.9034 - val_loss: 0.7794 - val_acc: 0.6972
Epoch 69/100
2257/2257 [==============================] - 0s 216us/step - loss: 0.2172 - acc: 0.9105 - val_loss: 0.7994 - val_acc: 0.6932
Epoch 70/100
2257/2257 [==============================] - 0s 199us/step - loss: 0.2126 - acc: 0.9123 - val_loss: 0.8242 - val_acc: 0.6972
Epoch 71/100
2257/2257 [==============================] - 0s 217us/step - loss: 0.2078 - acc: 0.9154 - val_loss: 0.9063 - val_acc: 0.6892
Epoch 72/100
2257/2257 [==============================] - 0s 218us/step - loss: 0.1979 - acc: 0.9158 - val_loss: 0.8509 - val_acc: 0.7012
Epoch 73/100
2257/2257 [==============================] - 0s 218us/step - loss: 0.1889 - acc: 0.9220 - val_loss: 0.8547 - val_acc: 0.7092
Epoch 74/100
2257/2257 [=======================